In [1]:
import os
from PIL import Image
import tenseal as ts
import torch
from torchvision import transforms
import numpy as np
import json

def encrypt_and_save_images(input_folder, output_folder, context, img_size=64):
    """
    Encrypts all images in `input_folder` and saves encrypted vectors to `output_folder`
    
    Args:
        input_folder: Path to folder containing chest X-rays (e.g., "./chest_xray/test/PNEUMONIA")
        output_folder: Where to save encrypted files (e.g., "./encrypted_test/PNEUMONIA")
        context: TenSEAL encryption context
        img_size: Image resize dimension (must match training)
    """
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    # Use the same transforms as training
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])
    
    # Process each image
    for img_name in os.listdir(input_folder):
        if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(input_folder, img_name)
            
            # Load and preprocess
            img = Image.open(img_path).convert('L')  # Grayscale
            img = transform(img)
            img_array = img.numpy().flatten()
            
            # Encrypt
            encrypted_img = ts.ckks_vector(context, img_array)
            
            # Save encrypted vector
            output_path = os.path.join(output_folder, f"{os.path.splitext(img_name)[0]}.enc")
            with open(output_path, 'wb') as f:
                f.write(encrypted_img.serialize())

In [2]:
class UltraSafeHEModel:
    def __init__(self, input_size, context):
        # Micro initial weights
        self.weights = ts.ckks_vector(context, np.random.randn(input_size) * 0.0001)
        self.bias = ts.ckks_vector(context, [0.0])
        self.context = context
    
    def ultra_safe_predict(self, x):
        """Prediction with quadruple safety checks"""
        try:
            # Scale down before operation
            scaled_x = x * 0.5
            scaled_weights = self.weights * 0.5
            return scaled_x.dot(scaled_weights) + (self.bias * 0.5)
        except:
            return ts.ckks_vector(self.context, [0.0])
    
    def ultra_safe_update(self, gradients):
        """Update that cannot possibly fail"""
        try:
            # Extreme scaling
            update = gradients * 0.00001  # 100,000x smaller
            self.weights = self.weights - update
        except:
            print("Used nuclear-safe fallback update")
            pass  # Skip update if even this fails

    def predict_single(self, encrypted_img, threshold=0.5):
        """Make prediction on one encrypted image"""
        try:
            output = self.ultra_safe_predict(encrypted_img)
            confidence = output.decrypt()[0]
            prediction = "PNEUMONIA" if confidence > threshold else "NORMAL"
            return prediction, float(confidence)
        except Exception as e:
            print(f"Prediction failed: {str(e)}")
            return "ERROR", 0.0





def load_he_model(load_dir="./pneumonia_he_model"):
    """Load the complete HE model package"""
    # 1. Load context
    with open(os.path.join(load_dir, "he_context.seal"), "rb") as f:
        context = ts.context_from(f.read())
    
    # 2. Load metadata
    with open(os.path.join(load_dir, "metadata.json"), "r") as f:
        metadata = json.load(f)
    
    # 3. Reconstruct model
    model = UltraSafeHEModel(
        input_size=metadata["input_size"],
        context=context
    )
    
    # 4. Load encrypted parameters
    with open(os.path.join(load_dir, "he_weights.seal"), "rb") as f:
        model.weights = ts.ckks_vector_from(context, f.read())
    
    with open(os.path.join(load_dir, "he_bias.seal"), "rb") as f:
        model.bias = ts.ckks_vector_from(context, f.read())
    
    return model, context




m_model, m_context = load_he_model()

# Paths (UPDATE THESE)
test_normal_dir = "./chest_xray/test/NORMAL"      # Folder with normal test images
test_pneumonia_dir = "./chest_xray/test/PNEUMONIA"  # Folder with pneumonia test images

# Output folders for encrypted images
encrypted_normal_dir = "./encrypted_test/NORMAL"
encrypted_pneumonia_dir = "./encrypted_test/PNEUMONIA"

# Encrypt and save
encrypt_and_save_images(test_normal_dir, encrypted_normal_dir, m_context)
encrypt_and_save_images(test_pneumonia_dir, encrypted_pneumonia_dir, m_context)

print("Encryption complete! Files saved to:")
print(f"- Normal X-rays: {encrypted_normal_dir}")
print(f"- Pneumonia X-rays: {encrypted_pneumonia_dir}")

FileNotFoundError: [Errno 2] No such file or directory: './pneumonia_he_model\\he_context.seal'